In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

!nvcc --version
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

%load_ext nvcc_plugin

In [ ]:
%%cu 
#include <stdio.h> // wariant GPU
#define N 100000

__global__ void add( int *a, int *b, int *c ) {
    int tid = blockIdx.x; // dodawanie na danych znajdujących się pod indeksem
    if (tid < N)
    c[tid] = a[tid] + b[tid];
}

int main( void ) {
    int a[N], b[N], c[N];
    int *dev_a, *dev_b, *dev_c;
    // alokacja pamięci na GPU
    cudaMalloc( (void**)&dev_a, N * sizeof(int) );
    cudaMalloc( (void**)&dev_b, N * sizeof(int) );
    cudaMalloc( (void**)&dev_c, N * sizeof(int) );
    // zapelnienie tablic a i b na CPU
    for (int i=0; i<N; i++) {
        a[i] = -i;
        b[i] = i * i;
    }
    // kopiowanie tablic a i b do GPU
    cudaMemcpy( dev_a, a, N * sizeof(int), cudaMemcpyHostToDevice );
    cudaMemcpy( dev_b, b, N * sizeof(int), cudaMemcpyHostToDevice );
    add<<<N,1>>>( dev_a, dev_b, dev_c );
    // kopiowanie tablicy c z GPU do CPU
    cudaMemcpy( c, dev_c, N * sizeof(int), cudaMemcpyDeviceToHost );
    // wyświetlenie wynikow
    for (int i=0; i<N; i++) {
        printf( "%d + %d = %d\n", a[i], b[i], c[i] );
    }
    // zwolnienie pamieci alokowanej na CPU
    cudaFree( dev_a );
    cudaFree( dev_b );
    cudaFree( dev_c );
    return 0;
}


In [31]:
%%cu
#include <stdio.h> //Wariant CPU
#define N 100000
void add( int *a, int *b, int *c ) {
    int tid = 0; // to jest CPU nr zero, a więc zaczynamy od zera
    while (tid < N) {
    c[tid] = a[tid] + b[tid];
    tid += 1; // mamy tylko jeden CPU, a więc zwiększamy o jeden
}

}
int main( void ) {
    int a[N], b[N], c[N];
    // zapelnianie tablic a i b za pomocą CPU
    for (int i=0; i<N; i++) {
    a[i] = -i;
    b[i] = i * i;
    }
    add( a, b, c );
    // wyswietlenie wynikow
    for (int i=0; i<N; i++) {
        printf( "%d + %d = %d\n", a[i], b[i], c[i] );
    }
    return 0;
}

Streaming output truncated to the last 5000 lines.
-95001 + 435255409 = 435160408
-95002 + 435445412 = 435350410
-95003 + 435635417 = 435540414
-95004 + 435825424 = 435730420
-95005 + 436015433 = 435920428
-95006 + 436205444 = 436110438
-95007 + 436395457 = 436300450
-95008 + 436585472 = 436490464
-95009 + 436775489 = 436680480
-95010 + 436965508 = 436870498
-95011 + 437155529 = 437060518
-95012 + 437345552 = 437250540
-95013 + 437535577 = 437440564
-95014 + 437725604 = 437630590
-95015 + 437915633 = 437820618
-95016 + 438105664 = 438010648
-95017 + 438295697 = 438200680
-95018 + 438485732 = 438390714
-95019 + 438675769 = 438580750
-95020 + 438865808 = 438770788
-95021 + 439055849 = 438960828
-95022 + 439245892 = 439150870
-95023 + 439435937 = 439340914
-95024 + 439625984 = 439530960
-95025 + 439816033 = 439721008
-95026 + 440006084 = 439911058
-95027 + 440196137 = 440101110
-95028 + 440386192 = 440291164
-95029 + 440576249 = 440481220
-95030 + 440766308 = 440671278
-95031 + 440956369 